# This notebook runs the Deep Gravity model on the yearly trade prediction dataset

In [1]:
import pandas as pd
import numpy as np
import os
import datetime
import sys
import tqdm

import random
import torch.utils.data.distributed
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
from ray import air, tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from functools import partial

import model_utils
from data_compiler import FlowDataset
from deepgravity import DeepGravity

sys.path.append('../trade_predictions')
import parameters

# random seeds
torch.manual_seed(parameters.seed)
np.random.seed(parameters.seed)
random.seed(parameters.seed)

torch_device = torch.device("cpu")


In [2]:
##############
# Load data
##############

nodes = pd.read_csv(parameters.node_path)
nodes_columns = parameters.node_features + [parameters.node_id] + [parameters.node_timestamp]
nodes = nodes[nodes_columns]

edges = pd.read_csv(parameters.edge_path)
edges_columns = parameters.flows_features + [parameters.flow_origin] + \
    [parameters.flow_destination] + [parameters.flows_timestamp] + \
        [parameters.flows_value]
edges = edges[edges_columns]

##############
# Initial cleaning
##############

nodes = nodes.fillna(0)

##############
# Create data objects
##############

columns = {'node_id': parameters.node_id,
           'node_timestamp': parameters.node_timestamp,
           'flow_origin': parameters.flow_origin,
           'flow_destination': parameters.flow_destination,
           'flows_timestamp': parameters.flows_timestamp,
           'flows_value':parameters.flows_value}

flow_data = FlowDataset(columns=columns,
                        unit = [parameters.flow_origin, parameters.flows_timestamp],
                        nodes=nodes,
                        edges=edges,)

# Create a list of FlowDataset objects
flow_data_chunked = flow_data.create_chunks(chunk_size=6)

# Add past values to each chunk
[flow_chunk.add_past_values(periods=parameters.lag_periods,
                            edge_columns = parameters.time_dependent_edge_columns,
                            node_columns = parameters.time_dependent_node_columns) for flow_chunk in tqdm.tqdm(flow_data_chunked)]

# Add target to each chunk
[flow_chunk.add_target_values() for flow_chunk in tqdm.tqdm(flow_data_chunked)]

# Create a list of FlowDataset objects
train_data_chunked = []
validation_data_chunked = []
test_data_chunked = []

for flow_data in tqdm.tqdm(flow_data_chunked):
    train_data, validation_data, test_data = flow_data.split_train_validate_test(validation_period = 1)
    train_data_chunked.append(train_data)
    validation_data_chunked.append(validation_data)
    test_data_chunked.append(test_data)

100%|██████████| 20/20 [00:00<00:00, 456.99it/s]


In [4]:
prediction_list = []
for chunk in range(len(train_data_chunked[:1])):
    # Set config where parameters are tuned
    config = {
        "lr": tune.loguniform(1e-4, 1e-1),
        "batch_size": tune.choice([2, 4, 8, 16]),
        "dim_hidden": tune.sample_from(lambda _: 2**np.random.randint(2, 6)),
        "dropout_p": tune.choice([0.25, 0.35, 0.45]),
        "num_layers": tune.choice([5, 10, 15]),
    }
    # Set scheduler
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=parameters.epochs,
        grace_period=1,
        reduction_factor=2)
    # Set reporter
    reporter = CLIReporter(
            # parameter_columns=["lr", "batch_size", "dim_hidden", "dropout_p", "num_layers"],
            metric_columns=["loss", "training_iteration"])
    # Run tuning
    result = tune.run(
        tune.with_parameters(model_utils.train_and_validate_deepgravity, train_data_chunked = train_data_chunked,
                validation_data_chunked = validation_data_chunked, chunk = chunk),
        resources_per_trial={"cpu": 4},
        config=config,
        num_samples=10,
        scheduler=scheduler,
        progress_reporter=reporter)

    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))

    input_dim = train_data_chunked[chunk].get_feature_dim()
    best_trained_model = DeepGravity(dim_input = input_dim,
                                    dim_hidden = best_trial.config["dim_hidden"],
                                    dropout_p = best_trial.config["dropout_p"],
                                    num_layers = best_trial.config["num_layers"],)

    best_checkpoint = result.get_best_checkpoint(trial=best_trial, metric="loss", mode="min")
    best_checkpoint_dir = best_checkpoint.to_directory(path=os.path.join(parameters.output_path, "best_checkpoints", "trade", str(chunk)))
    model_state, optimizer_state = torch.load(os.path.join(best_checkpoint_dir, f"checkpoint_{str(datetime.datetime.now()).replace(' ', '_')[:19]}"))
    best_trained_model.load_state_dict(model_state)

    test_data_loader = torch.utils.data.DataLoader(test_data_chunked[chunk], batch_size=4)
    model_utils.test(test_data_loader, best_trained_model, test_data_chunked[chunk], loss_fn = None, store_predictions=True)
    print("Finished prediction on test set")
    prediction_list.append(test_data_chunked[chunk].compile_predictions(columns_to_rename = parameters.columns_to_rename))
#pd.concat(prediction_list, axis=0).to_csv(f"{parameters.output_path}/prediction_{str(datetime.datetime.now()).replace(' ', '_')[:19]}.csv")

== Status ==
Current time: 2023-04-08 18:23:35 (running for 00:00:00.11)
Memory usage on this node: 5.7/8.0 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 4.0/4 CPUs, 0/0 GPUs, 0.0/2.65 GiB heap, 0.0/1.33 GiB objects
Result logdir: /Users/rgyuri/ray_results/train_and_validate_deepgravity_2023-04-08_18-23-35
Number of trials: 2/2 (1 PENDING, 1 RUNNING)
+--------------------------------------------+----------+-----------------+--------------+--------------+-------------+-----------+--------------+
| Trial name                                 | status   | loc             |   batch_size |   dim_hidden |   dropout_p |        lr |   num_layers |
|--------------------------------------------+----------+-----------------+--------------+--------------+-------------+-----------+--------------|
| train_and_validate_deepgravity_b5d43_00000 | RUNNING  | 127.0.0.1:22294 |            8 |           16 |  

Trial name,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,loss,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_and_validate_deepgravity_b5d43_00000,2023-04-08_18-24-04,True,,e50e7a39776d43dbb1d430181d3cb72d,"0_batch_size=8,dim_hidden=16,dropout_p=0.4500,lr=0.0749,num_layers=5",Gyorgys-MacBook-Pro.local,2,1.31703e+17,127.0.0.1,22294,True,14.0685,3.17818,14.0685,1680971044,0,,2,b5d43_00000,0.0116789
train_and_validate_deepgravity_b5d43_00001,2023-04-08_18-24-19,True,,e50e7a39776d43dbb1d430181d3cb72d,"1_batch_size=4,dim_hidden=32,dropout_p=0.4500,lr=0.0112,num_layers=15",Gyorgys-MacBook-Pro.local,2,4.32091e+13,127.0.0.1,22294,True,14.8015,0.916771,14.8015,1680971059,0,,2,b5d43_00001,0.0116789


(train_and_validate_deepgravity pid=22294) Finished training!
== Status ==
Current time: 2023-04-08 18:24:09 (running for 00:00:34.81)
Memory usage on this node: 6.2/8.0 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: -1.3170295056000614e+17 | Iter 1.000: -4.0227602027554703e+18
Resources requested: 4.0/4 CPUs, 0/0 GPUs, 0.0/2.65 GiB heap, 0.0/1.33 GiB objects
Result logdir: /Users/rgyuri/ray_results/train_and_validate_deepgravity_2023-04-08_18-23-35
Number of trials: 2/2 (1 RUNNING, 1 TERMINATED)
+--------------------------------------------+------------+-----------------+--------------+--------------+-------------+-----------+--------------+-------------+----------------------+
| Trial name                                 | status     | loc             |   batch_size |   dim_hidden |   dropout_p |        lr |   num_layers |        loss |   training_iteration |
|--------------------------------------------+------------+--------------

2023-04-08 18:24:19,700	INFO tune.py:798 -- Total run time: 44.61 seconds (44.57 seconds for the tuning loop).


== Status ==
Current time: 2023-04-08 18:24:19 (running for 00:00:44.59)
Memory usage on this node: 5.6/8.0 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: -6.587307983691627e+16 | Iter 1.000: -2.0114376190430828e+18
Resources requested: 0/4 CPUs, 0/0 GPUs, 0.0/2.65 GiB heap, 0.0/1.33 GiB objects
Result logdir: /Users/rgyuri/ray_results/train_and_validate_deepgravity_2023-04-08_18-23-35
Number of trials: 2/2 (2 TERMINATED)
+--------------------------------------------+------------+-----------------+--------------+--------------+-------------+-----------+--------------+-------------+----------------------+
| Trial name                                 | status     | loc             |   batch_size |   dim_hidden |   dropout_p |        lr |   num_layers |        loss |   training_iteration |
|--------------------------------------------+------------+-----------------+--------------+--------------+-------------+-----------+--------------+-

In [ ]:
#set([i[1] for i in train_data_chunked[0].data_dict.keys()])
#train_data_chunked[0].data_dict[(32, 1996)].columns
#data_loader = torch.utils.data.DataLoader(flow_data_chunked[0], batch_size=parameters.batch_size)
#for X, y in data_loader:
#    print(f"Shape of X : {X.shape}")
#    print(f"Shape of y: {y.shape} {y.dtype}")
#    break